In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Dataset\crypto_data\LTC-USD.csv", names= ['Time', 'Low', 'High','Open','Close','Volume'])

df.head(3)

,Time,Low,High,Open,Close,Volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799


In [3]:
main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD",'ETH-USD','BCH-USD']

In [4]:
for ratio in ratios:
    dataset = f"Dataset\crypto_data\{ratio}.csv"
    df = pd.read_csv(dataset, names= ['Time', 'Low', 'High','Open','Close','Volume'], index_col= 'Time')
    
    df.rename(columns= {"Close": f"{ratio}_close" , 'Volume': f'{ratio}_volume'},inplace= True)
    
    df = df[[f"{ratio}_close" , f'{ratio}_volume']]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

    

In [5]:
main_df.columns

Index(['BTC-USD_close', 'BTC-USD_volume', 'LTC-USD_close', 'LTC-USD_volume',
       'ETH-USD_close', 'ETH-USD_volume', 'BCH-USD_close', 'BCH-USD_volume'],
      dtype='object')

In [6]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [7]:
RATIO_TO_PREDICT = 'ETH-USD'
FUTURE_PERIOD_PREDICT = 3
SEQ_LEN = 60
EPOCHS = 10
BATCH_SIZE = 64
NAME = f"{RATIO_TO_PREDICT}-{SEQ_LEN} - SEQ - {FUTURE_PERIOD_PREDICT} - PRED - {int(time.time())}"

NameError: name 'time' is not defined

In [8]:
main_df['Future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(- FUTURE_PERIOD_PREDICT)

main_df[[f"{RATIO_TO_PREDICT}_close", 'Future']].head()

,ETH-USD_close,Future
Time,,
1528968660,NaN,485.75000
1528968720,486.01001,486.00000
1528968780,486.00000,486.00000
1528968840,485.75000,485.98999
1528968900,486.00000,485.98999


In [9]:
main_df['Target'] =  list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df[f'Future']))

In [10]:
main_df[[f'{RATIO_TO_PREDICT}_close', 'Future', 'Target']].head(10)

,ETH-USD_close,Future,Target
Time,,,
1528968660,NaN,485.75000,0
1528968720,486.01001,486.00000,0
1528968780,486.00000,486.00000,0
1528968840,485.75000,485.98999,1
1528968900,486.00000,485.98999,0
1528968960,486.00000,485.98999,0
1528969020,485.98999,485.98999,0
1528969080,485.98999,486.00000,1
1528969140,485.98999,486.00000,1


In [11]:
times = sorted(main_df.index.values)

last_5pct = times[-int(0.05 * len(times))]

In [12]:
last_5pct

1534922100

In [13]:
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

In [36]:
main_df

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,Future,Target
Time,,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,485.750000,0
1528968720,6487.379883,7.706374,96.660004,314.387024,486.010010,26.019083,870.859985,26.856577,486.000000,0
1528968780,6479.410156,3.088252,96.570000,77.129799,486.000000,8.449400,870.099976,1.124300,486.000000,0
1528968840,6479.410156,1.404100,96.500000,7.216067,485.750000,26.994646,870.789978,1.749862,485.989990,1
1528968900,6479.979980,0.753000,96.389999,524.539978,486.000000,77.355759,870.000000,1.680500,485.989990,0
...,...,...,...,...,...,...,...,...,...,...
1534921800,6686.250000,0.478039,57.509998,18.782650,285.230011,38.141129,551.299988,0.336000,286.000000,1
1534921860,6686.250000,0.440793,57.500000,8.449425,285.489990,17.549879,551.299988,0.010847,285.850006,1
1534921920,6686.250000,2.678847,57.509998,6.070000,285.739990,6.953944,551.299988,5.713912,285.739990,0


: 

In [15]:
from sklearn import preprocessing
from collections import deque
import random
import numpy as np

def preprocess_df(df):
    
    df= df.drop("Future",1)
    
    for col in df.columns:
        if col != 'Target':
            df[col] = df[col].pct_change()
            df.dropna(inplace= True)
            df[col] = preprocessing.scale(df[col].values)
            
        df.dropna(inplace =True)
        
    sequential_data = []
    prev_days = deque(maxlen = SEQ_LEN)
    
    for lists in df.values:
        prev_days.append([value for value in lists[:-1]])
        
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), lists[-1]])
            
            prev_days.clear()
            print(sequential_data)
            break
        
    #print(random.shuffle(sequential_data))
    #print(sequential_data)

In [22]:
preprocess_df(main_df)

[[array([[-2.51268250e-03, -7.89185022e-02, -5.40429356e-01,
        -1.17089656e-01,  4.60111811e-03, -4.78379997e-02,
         1.10617583e-02, -6.21709800e-03],
       [-2.29991651e-01,  3.40994132e-03,  4.04838613e-03,
        -1.18306852e-01,  4.60111811e-03, -4.53773491e-02,
         2.57552343e-01, -5.88380913e-03],
       [-8.30083053e-04, -7.40386327e-02,  4.04838613e-03,
        -7.29966458e-02,  2.12734814e-02, -3.29002556e-02,
         2.65415198e-01, -6.19975454e-03],
       [ 1.16298646e-01, -4.96264158e-02,  4.04838613e-03,
        -1.08241408e-01,  4.60111811e-03, -4.95302835e-02,
         4.55951014e-01, -5.89833729e-03],
       [ 9.36774676e-04, -8.20524538e-02,  4.04838613e-03,
        -4.85159270e-02,  4.60111811e-03, -3.29156249e-02,
         5.27186135e-01, -6.20610912e-03],
       [ 1.09471014e-01, -6.71763153e-02,  3.93210100e-01,
        -1.07576761e-01,  4.60111811e-03, -1.34541982e-02,
         3.10232629e-03, -6.10331066e-03],
       [ 3.26507676e-01, -2.9553

In [21]:
main_df.values

array([[6.48954980e+03, 5.87100000e-01, 9.65800020e+01, ...,
        5.67536100e+00, 4.85750000e+02, 0.00000000e+00],
       [6.48737988e+03, 7.70637400e+00, 9.66600040e+01, ...,
        2.68565770e+01, 4.86000000e+02, 0.00000000e+00],
       [6.47941016e+03, 3.08825200e+00, 9.65700000e+01, ...,
        1.12430000e+00, 4.86000000e+02, 0.00000000e+00],
       ...,
       [6.68625000e+03, 2.67884700e+00, 5.75099980e+01, ...,
        5.71391200e+00, 2.85739990e+02, 0.00000000e+00],
       [6.68625000e+03, 2.20156000e-01, 5.75099980e+01, ...,
                   nan, 2.85730011e+02, 0.00000000e+00],
       [6.68500000e+03, 6.40161100e+00, 5.75099980e+01, ...,
        2.64895000e-01, 2.85730011e+02, 0.00000000e+00]])

In [16]:
preprocess_df(main_df)

[[array([[-2.51268250e-03, -7.89185022e-02, -5.40429356e-01,
        -1.17089656e-01,  4.60111811e-03, -4.78379997e-02,
         1.10617583e-02, -6.21709800e-03],
       [-2.29991651e-01,  3.40994132e-03,  4.04838613e-03,
        -1.18306852e-01,  4.60111811e-03, -4.53773491e-02,
         2.57552343e-01, -5.88380913e-03],
       [-8.30083053e-04, -7.40386327e-02,  4.04838613e-03,
        -7.29966458e-02,  2.12734814e-02, -3.29002556e-02,
         2.65415198e-01, -6.19975454e-03],
       [ 1.16298646e-01, -4.96264158e-02,  4.04838613e-03,
        -1.08241408e-01,  4.60111811e-03, -4.95302835e-02,
         4.55951014e-01, -5.89833729e-03],
       [ 9.36774676e-04, -8.20524538e-02,  4.04838613e-03,
        -4.85159270e-02,  4.60111811e-03, -3.29156249e-02,
         5.27186135e-01, -6.20610912e-03],
       [ 1.09471014e-01, -6.71763153e-02,  3.93210100e-01,
        -1.07576761e-01,  4.60111811e-03, -1.34541982e-02,
         3.10232629e-03, -6.10331066e-03],
       [ 3.26507676e-01, -2.9553

In [24]:
print(x)

None


In [25]:
from sklearn import preprocessing
from collections import deque
import random
import numpy as np

def preprocess_df(df):
    
    df= df.drop("Future",1)
    
    for col in df.columns:
        if col != 'Target':
            df[col] = df[col].pct_change()
            df.dropna(inplace= True)
            df[col] = preprocessing.scale(df[col].values)
            
        df.dropna(inplace =True)
        
    sequential_data = []
    prev_days = deque(maxlen = SEQ_LEN)
    
    for lists in df.values:
        prev_days.append([value for value in lists[:-1]])
        
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), lists[-1]])
            
            prev_days.clear()
        
    random.shuffle(sequential_data)
    
    buys =[]
    sells =[]
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])   
            
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower_length = min(len(buys), len(sells))
    
    buys = buys[:lower_length]
    sells = sells[:lower_length]
    
    sequential_data = buys + sells
    random.shuffle(sequential_data)  
    
    X =[]
    y= []
    
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)   
        
        
        
    return np.array(X) , np.array(y)
        
            
            
    
    


In [26]:
train_x, train_y = preprocess_df(main_df)
validation_train_x, validation_train_y = preprocess_df(validation_main_df)

In [35]:
len(train_x)

1226

In [27]:
print(f"Train data: {len(train_x)} Validation: {len(validation_train_x)}")
print(f"Don't buys: {train_y.count(0)}, Buys: {train_y.count(1)}")
print(f"Validation Don't Buys: {validation_train_y.count(0)}, Buys: {validation_train_y.count(1)} ")

Train data: 1226 Validation: 52


AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [28]:
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LSTM,  BatchNormalization, LSTM
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint


In [74]:
model = Sequential()

model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences= True))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences= True))
model.add(Dropout(0.1))
model.add(BatchNormalization())


model.add(LSTM(128, input_shape= (train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation= 'relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation= 'softmax') )

opt = tf.keras.optimizers.Adam(learning_rate= 0.001, decay= 1e-6)

model.compile(loss='sparse_categorical_crossentropy', optimizer= opt, metrics=['accuracy']) 

In [75]:
tensorBoard = TensorBoard(log_dir= f'logs/{NAME}')

filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"

checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_accuracy',
                                                      verbose =1, save_best_only =True, mode ='max'))

In [76]:
history = model.fit(
    train_x, train_y,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = (validation_train_x, validation_train_y),
    callbacks =[tensorBoard, checkpoint]
)




Epoch 1/10
19/19 [==============================] - ETA: 0s - loss: 0.9541 - accuracy: 0.5206

INFO:tensorflow:Assets written to: models\RNN_Final-01-0.500.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-01-0.500.model\assets


19/19 [==============================] - 32s 1s/step - loss: 0.9541 - accuracy: 0.5206 - val_loss: 0.6973 - val_accuracy: 0.5000
Epoch 2/10
19/19 [==============================] - ETA: 0s - loss: 0.8069 - accuracy: 0.5455

INFO:tensorflow:Assets written to: models\RNN_Final-02-0.483.model\assets


...

INFO:tensorflow:Assets written to: models\RNN_Final-09-0.500.model\assets


19/19 [==============================] - 43s 2s/step - loss: 0.6019 - accuracy: 0.6630 - val_loss: 0.7374 - val_accuracy: 0.5000
Epoch 10/10
19/19 [==============================] - ETA: 0s - loss: 0.6067 - accuracy: 0.6690

INFO:tensorflow:Assets written to: models\RNN_Final-10-0.448.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-10-0.448.model\assets


19/19 [==============================] - 42s 2s/step - loss: 0.6067 - accuracy: 0.6690 - val_loss: 0.7386 - val_accuracy: 0.4483


In [261]:
for lists in train_x: 
    for sub_lists in lists:
        print(sub_lists)
        
    break

[ 0.10565824 -0.39615151  0.2596697  -0.35394784 -0.33650175 -0.4685614
  0.69206188 -0.64325577]
[ 0.23389628 -0.03274868  2.05540937  0.21725638  1.02911025  3.13154496
 -0.66897842 -0.13093787]
[ 0.10565824 -0.34109093 -1.11996198 -0.35137633 -0.17291929 -0.56039221
 -2.18210688 -0.64954548]
[ 0.10565824  4.08087821  1.36369278 -0.28320193  3.50991041  1.24840551
  2.28339319  2.99019415]


In [269]:
train_x

array([[[-3.15769993e-01,  3.59694894e-02, -9.04724923e-02, ...,
         -2.39072359e-02, -1.09708897e+00, -5.77000783e-03],
        [-8.40597192e-02, -7.17744283e-02, -1.85168529e-01, ...,
         -5.02012411e-02,  6.96080841e-02, -5.98935507e-03],
        [-8.30083053e-04, -8.66368943e-02,  4.04838613e-03, ...,
         -4.44161805e-02,  3.10232629e-03, -6.22463642e-03],
        ...,
        [ 1.13807701e-01,  2.14949059e-02,  3.79707370e-01, ...,
         -4.35119580e-02,  2.01926928e-01, -6.22422956e-03],
        [-2.55255452e-03, -7.56828230e-02,  4.04838613e-03, ...,
         -4.94944854e-02,  5.80400781e-01, -6.19704013e-03],
        [-8.30083053e-04,  4.41877052e-01, -1.12238343e+00, ...,
         -4.17099787e-02, -6.77719160e-01, -5.76037741e-03]],

       [[-1.26225276e+00, -8.63295984e-02, -4.82355471e-01, ...,
         -4.31581611e-02, -3.59760900e-01, -6.19107993e-03],
        [-9.95842660e-02, -6.70990217e-02, -7.74597456e-01, ...,
         -4.61733425e-02, -1.16656057e

In [174]:
train_x[:61]

array([[[-8.07556189e-01, -3.55502191e-02, -9.38807827e-01, ...,
         -1.91583263e-02, -1.73930993e+00, -5.25736392e-03],
        [-2.64525831e+00,  1.98470436e-01, -1.20972043e+00, ...,
         -4.71804035e-03, -2.95308459e+00, -6.18206240e-03],
        [-5.90628334e-01, -8.82358402e-02, -6.71346613e-01, ...,
         -5.05294761e-02,  1.87890090e+00, -6.20817131e-03],
        ...,
        [ 8.74817811e-01, -2.41377792e-02,  2.72245802e-01, ...,
         -3.42210248e-02,  3.10232629e-03, -6.16650644e-03],
        [-8.30083053e-04, -8.44357666e-02,  1.38119289e-01, ...,
         -4.43152204e-02,  3.10232629e-03, -6.09780985e-03],
        [-8.30083053e-04,  1.50196452e-01,  1.74627011e+00, ...,
         -4.51368565e-02,  3.10232629e-03, -6.21182542e-03]],

       [[-2.48862792e-01, -7.02702888e-02,  4.04838613e-03, ...,
         -2.59628778e-02, -5.37709987e-01, -6.20452346e-03],
        [-8.55822784e-01, -2.99290901e-02, -8.01038896e-01, ...,
         -4.04547050e-02, -1.84517209e

In [137]:
#preprocess_df(main_df)

In [262]:
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,Future,Target
Time,,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,96.500000,0
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577,96.389999,0
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300,96.519997,0
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862,96.440002,0
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500,96.470001,1


In [231]:
lis = [1,2,3,4,5,6,7,8,9,10]
lis[:3]
#print(lis)
#
#lis.clear()
#print(lis)

[1, 2, 3]

In [213]:
lis1 = deque(maxlen= 3)

for items in lis: 
    lis1.append(items)
    
    if len(lis1) == 2:
        print("Sab AR")

lis1

Sab AR


deque([8, 9, 10])

In [251]:
sequential_data = []
prev_days = deque(maxlen = SEQ_LEN)
    
for lists in main_df.values[:10]:
    prev_days.append([value for value in lists[:-1]])
    
    if len(prev_days) == SEQ_LEN:
        sequential_data.append([np.array(prev_days), lists[-1]])
        
        #prev_days.clear()
        
        
sequential_data

[[array([[6.48954980e+03, 5.87100000e-01, 9.65800020e+01, 9.64720000e+00,
                     nan,            nan, 8.71719971e+02, 5.67536100e+00,
          9.65000000e+01],
         [6.48737988e+03, 7.70637400e+00, 9.66600040e+01, 3.14387024e+02,
          4.86010010e+02, 2.60190830e+01, 8.70859985e+02, 2.68565770e+01,
          9.63899990e+01],
         [6.47941016e+03, 3.08825200e+00, 9.65700000e+01, 7.71297990e+01,
          4.86000000e+02, 8.44940000e+00, 8.70099976e+02, 1.12430000e+00,
          9.65199970e+01],
         [6.47941016e+03, 1.40410000e+00, 9.65000000e+01, 7.21606700e+00,
          4.85750000e+02, 2.69946460e+01, 8.70789978e+02, 1.74986200e+00,
          9.64400020e+01]]),
  0.0],
 [array([[6.48737988e+03, 7.70637400e+00, 9.66600040e+01, 3.14387024e+02,
          4.86010010e+02, 2.60190830e+01, 8.70859985e+02, 2.68565770e+01,
          9.63899990e+01],
         [6.47941016e+03, 3.08825200e+00, 9.65700000e+01, 7.71297990e+01,
          4.86000000e+02, 8.44940000e+00,

In [203]:
lis1.clear()

In [204]:
len(lis1)

0

In [ ]:
import spacy